In [ ]:
%%capture
!pip install ultralytics
import ultralytics
ultralytics.checks()

## Setting Variables

In [2]:
# video sequence name - "simple_mid", "simple_ben", "difficult_mid", "difficult_ben"
vid_seq_name = "difficult_mid"

#path to model and tracker files
model_path = '/Users/eliu/Desktop/mbari_452k_yolov8.pt'
tracker_path = "./bytetrack_kwalz_settings.yaml" # default bytetrack: "bytetrack.yaml"

#set video paths
video_paths = {
    "simple_mid": "./videos/simple_mid/Midwater_simple_V4455_20221130T192123Z_prores.mov",
    "simple_ben": "./videos/simple_ben/10s_Benthic_2_V4289_20200729T185027Z.mov",
    "difficult_mid": "./videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov",
    "difficult_ben": "./videos/difficult_ben/Benthic_1_V4277_20200219T211238Z.mov"
    #to add video sequences, add path here
}
vid_path = video_paths.get(vid_seq_name)
if vid_path is None: raise ValueError(f"invalid name: {vid_seq_name}")

## Defining Functions

In [3]:
import os
import glob
import cv2 

#convert from yolo txt file format to motchallenge file format 
def yolo_to_mot(yolo_labels, output_mot, vid_path):

    # get frame size from video
    cap = cv2.VideoCapture(vid_path)
    ret, frame = cap.read()
    if not ret:
        raise ValueError("Failed to read video")
    img_height, img_width = frame.shape[:2]
    cap.release()
    
    # unnormalize labels
    count = 0
    frame_files = sorted(glob.glob(os.path.join(yolo_labels, "*.txt")))
    with open(output_mot, "w") as out_file:
        for file in frame_files:
            name = os.path.basename(file).split(".")[0]
            frame_idx = int(name.split("_")[-1])
            with open(file, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 6:
                        print(f"Skipping line in {file}: {line.strip()}")
                        count = count + 1
                        continue
                    class_id, x_center, y_center, width, height, track_id = map(float, line.strip().split())
                    #calculate
                    x = (x_center - width / 2) * img_width
                    y = (y_center - height / 2) * img_height
                    w = width * img_width
                    h = height * img_height

                    out_file.write(f"{frame_idx}, {int(track_id)}, {x:.2f}, {y:.2f}, {w:.2f}, {h:.2f}, 1, 1, -1\n")
    print("number of skipped lines: ", count)

#adding track id to yolo txt files
def add_track_id(input_folder, output_folder):
    #get all name files
    name_files = [f for f in os.listdir(input_folder) if f.endswith("_name.txt")]
    for name_file in name_files:
        #paths
        base_file = name_file.replace("_name", "")
        name_file_path = os.path.join(input_folder, name_file)
        base_file_path = os.path.join(input_folder, base_file)
        output_file_path = os.path.join(output_folder, base_file)

        # reading files
        with open(name_file_path, "r") as f_name, open(base_file_path, "r") as f_base:
            name_lines = f_name.readlines()
            base_lines = f_base.readlines()

        #make new files
        with open(output_file_path, "w") as f_out:
            for base_line, name_line in zip(base_lines, name_lines):
                if base_line.strip() and name_line.strip():
                    # get track id
                    track_id = int(name_line.strip().rsplit("-", 1)[-1])
                    # combine line + track id
                    f_out.write(base_line.strip() + f" {track_id}\n")


## Model + Tracker
Running model and tracker on video

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO(model_path)
results = model.track(source=vid_path,
                tracker=tracker_path,
                line_width=1,
                agnostic_nms=True,
                #save_crop=True,
                #save_conf=True,
                save_txt=True,
                save=True,
                #save_frames=True,
                conf=0.001,
                iou=0.2,
                # augment=True,
                # half=True,
                device='mps',
                #vid_stride=60,
                #show=True,
                #visualize=True,
                stream=True,
                imgsz=1280
                )

# want the save_dir path, so loop until last result
last_result = None
for r in results:
    last_result = r 

# get the label path from the final result
labels_dir = last_result.save_dir
yolo_labels = labels_dir + '/labels' # where the saved labels are
output_mot = f"./data/predictions/mot_challenge/{vid_seq_name}.txt" #where output motchallenge file will go

yolo_to_mot(yolo_labels, output_mot, vid_path)

print("\nModel detections have been saved to ", output_mot)
with open(output_mot, 'r') as f:
    contents = f.read()
    print("Contents of model detections in MOT format:\n")
    print(contents)


video 1/1 (frame 1/600) /Users/eliu/Documents/benchmark_eval/videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov: 736x1280 1 Aegina, 5 Eusergestes similiss, 6 Merluccius productuss, 1 Pyrosoma, 214.0ms
video 1/1 (frame 2/600) /Users/eliu/Documents/benchmark_eval/videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov: 736x1280 4 Eusergestes similiss, 6 Merluccius productuss, 2 Pyrosomas, 134.5ms
video 1/1 (frame 3/600) /Users/eliu/Documents/benchmark_eval/videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov: 736x1280 5 Eusergestes similiss, 6 Merluccius productuss, 1 Pyrosoma, 134.6ms
video 1/1 (frame 4/600) /Users/eliu/Documents/benchmark_eval/videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.mov: 736x1280 4 Eusergestes similiss, 6 Merluccius productuss, 2 Pyrosomas, 134.4ms
video 1/1 (frame 5/600) /Users/eliu/Documents/benchmark_eval/videos/difficult_mid/Midwater_difficult_V4432_20220914T160635Z_prores.

## Ground Truth
Adding track id to exported ground truth YOLO txt files

In [ ]:
# GROUND TRUTH
# adding track id to exported gt yolo txt files
import os

#paths - should not have to change unless directory structure changes
input_path = "./temp/labels"
output_path = "./temp/labels_w_trackid"
os.makedirs(output_path, exist_ok=True)

# add track id to yolo txt files
add_track_id(input_path, output_path)
print("Track ID was successfully added")

# gt paths
yolo_labels = "./temp/labels_w_trackid"
output_mot = f"./data/gt/{vid_seq_name}/gt/gt.txt"

# converting format
yolo_to_mot(yolo_labels, output_mot, vid_path)
print("Ground truth has been saved to ", output_mot)
with open(output_mot, 'r') as f:
    contents = f.read()
    print("Contents of GT file in MOT format:\n")
    print(contents)


Track ID was successfully added
number of skipped lines:  0
Ground truth has been saved to  ./data/gt/difficult_mid/gt/gt.txt
Contents of output MOT file:

1, 510, 949.00, 228.00, 56.00, 50.00, 1, 1, -1
1, 503, 149.00, 198.00, 62.00, 45.00, 1, 1, -1
1, 501, 102.00, 717.00, 27.00, 25.00, 1, 1, -1
1, 500, 305.00, 356.00, 26.00, 17.00, 1, 1, -1
1, 31, 330.00, 374.00, 29.00, 26.00, 1, 1, -1
1, 217, 514.00, 218.00, 16.00, 12.00, 1, 1, -1
1, 156, 773.00, 186.00, 18.00, 16.00, 1, 1, -1
1, 102, 164.00, 261.00, 51.00, 37.00, 1, 1, -1
1, 111, 1614.00, 84.00, 48.00, 63.00, 1, 1, -1
1, 60, 459.00, 142.00, 44.00, 40.00, 1, 1, -1
1, 48, 1061.00, 85.00, 45.00, 43.00, 1, 1, -1
1, 45, 759.00, 217.00, 25.00, 21.00, 1, 1, -1
1, 39, 1749.00, 328.00, 80.00, 86.00, 1, 1, -1
1, 37, 838.00, 139.00, 33.00, 39.00, 1, 1, -1
1, 36, 1471.00, 462.00, 26.00, 32.00, 1, 1, -1
1, 35, 515.00, 306.00, 46.00, 24.00, 1, 1, -1
1, 32, 246.16, 434.00, 36.84, 47.05, 1, 1, -1
1, 30, 1694.00, 686.00, 21.00, 51.00, 1, 1, -1
1, 28